In [32]:
#from causalml.dataset import synthetic_data
import seaborn as sns
import matplotlib
#from causalml.inference.meta import BaseSClassifier, BaseTClassifier, BaseXClassifier
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.base import clone
import random
from scipy.stats import norm
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier
import itertools
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize
import pandas as pd
from sklift.metrics import qini_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn import preprocessing
import kuplift as kp
import warnings
import importlib
import subprocess
from sklearn.metrics import mean_squared_error
import numpy as np
import sys
from sklearn.model_selection import train_test_split
from matplotlib.colors import LinearSegmentedColormap
from tqdm import tqdm
import time
import ray
from modèles import *
from évaluation import *
from generate_synth import *
import logging
import gc
import re
warnings.filterwarnings('ignore')
from causalml.dataset import synthetic_data
import matplotlib
from causalml.inference.meta import BaseSClassifier, BaseTClassifier, BaseXClassifier
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [33]:
ray.shutdown()

In [ ]:
dict_rmse_1={'S_learner_XGB' : [], 'T_learner_XGB' : [], 'Baseline average' : [], 'Politique optimale' : []}

dict_rmse_2={'S_learner_XGB' : [], 'T_learner_XGB' : [], 'Baseline average' : [], 'Politique optimale' : []}

dict_expected={'S_learner_XGB' : [],'T_learner_XGB' : [], 'Baseline average' : [], 'Politique optimale' : []}

EXPECTED_S_XGB= {}

EXPECTED_T_XGB= {}

def get_max_index(array):
    # Crée un tableau pour stocker les indices des valeurs maximales

    try:
        array = array.astype(float)
    except ValueError:
        raise ValueError("Le tableau contient des valeurs non numériques qui ne peuvent pas être converties.")

    result = []
    
    # Parcourir chaque individu (chaque ligne du tableau)
    for row in array:
        # Vérifie si toutes les valeurs de la ligne sont inférieures ou égales à zéro
        if np.all(row <= 0):
            result.append(0)  # Si aucune valeur n'est supérieure à 0, ajouter 0
        else:
            # Sinon, trouve l'indice de la valeur maximale dans la ligne
            max_index = np.argmax(row)
            result.append(max_index + 1)  # Ajouter +1 pour ne pas avoir 0 comme index
            
    return np.array(result)

@ray.remote
class ResultActor:
    def __init__(self):
        self.data = {}

    def update(self, percent, value):
        self.data[percent] = value

    def get_data(self):
        return self.data


@ray.remote
class SharedState:
    def __init__(self):
        self.expected_s_xgb = {}
        self.expected_t_xgb = {}
        self.EXPECTED_TRUE= {}
        self.EXPECTED_random= {}
        self.EXPECTED_optimal= {}
        self.EXPECTED_zero= {}
        self.EXPECTED_average= {}
        self.nb_bons_t_S= {}
        self.nb_bons_t_T= {}
        self.nb_bons_t_true= {}
        self.nb_bons_t_random= {}
        self.nb_bons_t_average= {}
        self.rmse_1_S = {}
        self.rmse_1_T = {}
        
    def update(self, percent, s_value, t_value,expected_true,expected_random,expected_optimal,expected_zero,rmse_1_S,rmse_1_T):
        self.expected_s_xgb[percent] = s_value
        self.expected_t_xgb[percent] = t_value
        self.EXPECTED_TRUE[percent] = expected_true
        self.EXPECTED_random[percent] = expected_random
        self.EXPECTED_optimal[percent] = expected_optimal
        self.EXPECTED_zero[percent] = expected_zero
        #self.EXPECTED_average[percent] = expected_average
        #self.nb_bons_t_S[percent] = nb_bons_t_S
        #self.nb_bons_t_T[percent] = nb_bons_t_T
        #self.nb_bons_t_true[percent] = nb_bons_t_true
        #self.nb_bons_t_random[percent] = nb_bons_t_random
        #self.nb_bons_t_average[percent] = nb_bons_t_average
        self.rmse_1_S[percent] = rmse_1_S
        self.rmse_1_T[percent] = rmse_1_T

    def get_expected_s_xgb(self):
        return self.expected_s_xgb

    def get_expected_t_xgb(self):
        return self.expected_t_xgb

    def get_expected_true(self):
        return self.EXPECTED_TRUE

    def get_expected_random(self):
        return self.EXPECTED_random

    def get_expected_opti(self):
        return self.EXPECTED_optimal

    def get_expected_zero(self):
        return self.EXPECTED_zero

    def get_expected_average(self):
        return self.EXPECTED_average

    def get_nb_bon_t_S(self):
        return self.nb_bons_t_S

    def get_nb_bon_t_T(self):
        return self.nb_bons_t_T

    def get_nb_bon_t_true(self):
        return self.nb_bons_t_true

    def get_nb_bon_t_random(self):
        return self.nb_bons_t_random

    def get_nb_bon_t_average(self):
        return self.nb_bons_t_average
        
    def get_rmse_S(self):
        return self.rmse_1_S

    def get_rmse_T(self):
        return self.rmse_1_T


@ray.remote
def iteration(data,percent,shared_state_actor,decoupage=4,nb_per_t=1000,NRA='NRA',valeur_min_int=1,valeur_max_int = 4,nb_inter = 8,X1='X1',X2='X2'):
    #data = ray.get(data_ref)

    start_time = time.time()
   
    nombre_valeurs_uniques_T = data['T'].nunique()
    df_grand = pd.DataFrame()
    seed=42
    random.seed(seed)
    for i in range(nombre_valeurs_uniques_T):
        motif_random = generer_motif_random(valeur_min_int, valeur_max_int, nb_inter)
        motif = motif_to_int(motif_random,decoupage=decoupage)
        dataset_T = data[data['T'] == i]
        #dataset_T[NRA] = 0
        dataset_T.loc[:, NRA] = 0
        for index, row in dataset_T.iterrows():
            for i in motif:
                if (i[0][0] <= row[X1] <= i[0][1]) & (i[1][0] <= row[X2] <= i[1][1]):
                    #dataset_T.at[index, NRA] = 1
                    dataset_T.loc[index, NRA] = 1
        data1 = dataset_T.head(nb_per_t)
        data1 = drop_nra(data1,k=percent,NRA='NRA')
        data2 = dataset_T.iloc[nb_per_t:]
        data_NRA_0 = data2[data2['NRA'] == 0]
        data_remplace = data_NRA_0.head(int(nb_per_t - len(data1)))
        data_percent_NRA = pd.concat([data1, data_remplace], ignore_index=True)
        df_grand = pd.concat([df_grand, data_percent_NRA], ignore_index=True)
    
    del data1, data2, data_NRA_0, data_remplace, data_percent_NRA
    gc.collect()

    valeurs_unique = df_grand['T'].unique()
    
    colonnes_a_conserver = ['X1', 'X2', 'T']
    X_train = df_grand[colonnes_a_conserver]
    colonnes_a_conserver = ['X1', 'X2', 'T', 'Y']
    X_train_t = df_grand[colonnes_a_conserver]
    colonnes_a_conserver = ['X1', 'X2', 'T']
    X_test = data[colonnes_a_conserver]
    X_test = X_test.drop(X_train.index)
    colonnes_a_conserver = ['Y']
    y_train = df_grand[colonnes_a_conserver]
    #train_final_filtered = df_grand.loc[:, ['X1', 'X2', 'T', 'Y']]
    colonnes_a_conserver = ['X1', 'X2', 'T', 'Y']
    X_test_t = data[colonnes_a_conserver]
    X_test_t = X_test_t.drop(X_train.index)
    test_all = data.copy()
    test_all = test_all.drop(X_train.index)
    random_uplift = expected_to_uplift(random_expected_outcome(X_test_t, 'T'))
    random_policy = uplift_to_policy(random_uplift)
    

    
    X_train = df_grand[['X1', 'X2']]
    treatment_train = df_grand['T']
    y_train = df_grand['Y']

    X_test = data[['X1', 'X2']]
    X_test = X_test.drop(X_train.index)
    treatment_test = data['T']
    treatment_test = treatment_test.drop(X_train.index)
    y_test = data['Y']
    y_test = y_test.drop(X_train.index)
   
    data_test_t=data.copy()
    column_names = data_test_t.columns.tolist()
    data_test_t['policy'] = data_test_t.apply(determine_value, axis=1)
    tau_optimal_dict = {}
    #colonnes_E_y = [col for col in RMSE_test.columns if col.startswith('E_y|T') and not col.endswith('no_rand')]

    #optimal_policy = colonnes_max.apply(lambda x: int(x.split('_')[1]))
    optimal_policy = data_test_t['policy']
    data_test_t['worst_policy'] = data_test_t.apply(determine_value_2, axis=1)
    worst_policy = data_test_t['worst_policy'].to_numpy()
    zero_policy=[0]*len(random_policy)
    
    end_time = time.time()
    elapsed_time = end_time - start_time

    start_time = time.time()

    #X Learner (XGB)

    x_learner = BaseXClassifier(treatment_outcome_learner=XGBClassifier(random_state=42), 
                            treatment_effect_learner=XGBRegressor(random_state=42),
                            control_outcome_learner=XGBClassifier(random_state=42),
                            control_effect_learner=XGBRegressor(random_state=42),
                            control_name=0)

    x_learner.fit(X_train, treatment_train, y_train)


    uplift_S_learner_XGB = x_learner.predict(X_test)
    predict_policy_S_learner_XGB = get_max_index(uplift_S_learner_XGB)

    #predict_worst_policy_X_learner_XGB = s_learner_model_XGB .predict_worst_policy(X_test)
    

    end_time = time.time()
    elapsed_time = end_time - start_time
    start_time = time.time()
   
    #X learner (RandomForest)

    x_learner = BaseXClassifier(
    treatment_outcome_learner=RandomForestClassifier(random_state=42),
    treatment_effect_learner=RandomForestRegressor(random_state=42),
    control_outcome_learner=RandomForestClassifier(random_state=42),
    control_effect_learner=RandomForestRegressor(random_state=42),
    control_name=0)



    x_learner.fit(X_train, treatment_train, y_train)
    uplift_T_learner_XGB = x_learner.predict(X_test)
    predict_policy_T_learner_XGB = get_max_index(uplift_T_learner_XGB)

    end_time = time.time()
    elapsed_time = end_time - start_time
    start_time = time.time()
    
    expected_outcome_S_learner_XGB=expected_outcome(X_test_t,'T','Y',policy=predict_policy_S_learner_XGB)
    expected_outcome_T_learner_XGB=expected_outcome(X_test_t,'T','Y',policy=predict_policy_T_learner_XGB)
    expected_outcome_true_policy=expected_outcome(X=X_test_t,traitement = 'T',outcome = 'Y')
    expected_outcome_random_policy=expected_outcome(X=X_test_t,traitement = 'T',outcome = 'Y',policy=random_policy)
    expected_outcome_optimal=expected_outcome(X=X_test_t,traitement = 'T',outcome = 'Y',policy=optimal_policy)
    expected_outcome_zero_policy=expected_outcome(X_test_t,'T','Y',policy=zero_policy)

    politiques = ['S-Learner_XGB'
                'T-Learner_XGB', 
                'True Policy', 'Random Policy', 'Optimal Policy','Zero policy', 'S_Learner MLP2','Baseline average']

    values = [expected_outcome_S_learner_XGB, 
             expected_outcome_T_learner_XGB,  
            expected_outcome_true_policy, expected_outcome_random_policy,expected_outcome_optimal,expected_outcome_zero_policy]

    politiques, values = zip(*sorted(zip(politiques, values), key=lambda x: x[1], reverse=False))

    bar_width = 0.5
    bar_positions = np.arange(len(politiques))
    brilliant_color = (1.0, 0.8, 0.0)

    tau_1_0 = test_all['tau_1_0'].values
    tau_2_0 = test_all['tau_2_0'].values

    tau_columns = [col for col in test_all.columns if col.startswith('tau_')]
    num_tau_columns = len(tau_columns)
    rmse_list = []
    for i, tau_col in enumerate(tau_columns):
        true_values = test_all[tau_col].values
        
        pred_value = [sublist[i] for sublist in uplift_S_learner_XGB]
        rmse = calculate_rmse(true_values, pred_value)
        rmse_list.append(rmse)

    # Calculer la RMSE moyenne
    average_rmse = np.mean(rmse_list)
    tau_1_0_S_learner_XGB = [item[0] for item in uplift_S_learner_XGB]
    tau_2_0_S_learner_XGB = [item[1] for item in uplift_S_learner_XGB]
    rmse_tau_1_0_S_learner_XGB = average_rmse
    rmse_tau_2_0_S_learner_XGB = np.sqrt(mean_squared_error(tau_2_0, tau_2_0_S_learner_XGB))

    rmse_list = []
    for i, tau_col in enumerate(tau_columns):
        true_values = test_all[tau_col].values
        
        pred_value = [sublist[i] for sublist in uplift_T_learner_XGB]
        rmse = calculate_rmse(true_values, pred_value)
        rmse_list.append(rmse)
    average_rmse = np.mean(rmse_list)

    tau_1_0_T_learner_XGB = [item[0] for item in uplift_T_learner_XGB]
    tau_2_0_T_learner_XGB = [item[1] for item in uplift_T_learner_XGB]
    rmse_tau_1_0_T_learner_XGB = average_rmse
    rmse_tau_2_0_T_learner_XGB = np.sqrt(mean_squared_error(tau_2_0, tau_2_0_T_learner_XGB))

    tau_1_0_optimal = test_all.apply(lambda row: row['E_y|T1_no_rand'] - row['E_y|T0_no_rand'], axis=1)
    tau_2_0_optimal = test_all.apply(lambda row: row['E_y|T2_no_rand'] - row['E_y|T0_no_rand'], axis=1)
    uplift_optimal = [[diff, value] for diff, value in zip(tau_1_0_optimal, tau_2_0_optimal)]
    rmse_tau_1_0_optimal = np.sqrt(mean_squared_error(tau_1_0, tau_1_0_optimal))
    rmse_tau_2_0_optimal = np.sqrt(mean_squared_error(tau_2_0, tau_2_0_optimal))


    end_time = time.time()
    elapsed_time = end_time - start_time

    
    methods = ['S_learner_XGB', 'T_learner_XGB', 'Optimal','Baseline average']

    rmse_tau_1_0_list = [rmse_tau_1_0_S_learner_XGB, rmse_tau_1_0_T_learner_XGB, rmse_tau_1_0_optimal]

    rmse_tau_2_0_list = [rmse_tau_2_0_S_learner_XGB, rmse_tau_2_0_T_learner_XGB, rmse_tau_2_0_optimal]

    pairs_tau_1_0 = zip(methods, rmse_tau_1_0_list)
    pairs_tau_2_0 = zip(methods, rmse_tau_2_0_list)
    sorted_pairs_tau_1_0 = sorted(pairs_tau_1_0, key=lambda x: x[1])
    sorted_pairs_tau_2_0 = sorted(pairs_tau_2_0, key=lambda x: x[1])
    sorted_methods_tau_1_0, sorted_rmse_tau_1_0 = zip(*sorted_pairs_tau_1_0)
    sorted_methods_tau_2_0, sorted_rmse_tau_2_0 = zip(*sorted_pairs_tau_2_0)
    OPTI = optimal_policy[:1000]
    
    optimal_policy_df = pd.DataFrame(OPTI, columns=['policy'])
    optimal_policy_random = optimal_policy_df.sample(n=100, random_state=42)

    true_expected_outcome_S_learner_XGB = true_expected_outcome(test_all,predict_policy_S_learner_XGB)
    true_expected_outcome_T_learner_XGB = true_expected_outcome(test_all,predict_policy_T_learner_XGB)
    true_expected_outcome_optimal = true_expected_outcome(test_all,optimal_policy)
    
    dict_rmse_1['S_learner_XGB'].append(rmse_tau_1_0_S_learner_XGB)
    dict_rmse_1['T_learner_XGB'].append(rmse_tau_1_0_T_learner_XGB)
    dict_rmse_1['Politique optimale'].append(rmse_tau_1_0_optimal)
    dict_rmse_2['S_learner_XGB'].append(rmse_tau_2_0_S_learner_XGB)
    dict_rmse_2['T_learner_XGB'].append(rmse_tau_2_0_T_learner_XGB)
    dict_rmse_2['Politique optimale'].append(rmse_tau_2_0_optimal)
    dict_expected['S_learner_XGB'].append(expected_outcome_S_learner_XGB)
    dict_expected['T_learner_XGB'].append(expected_outcome_T_learner_XGB)
    dict_expected['Politique optimale'].append(expected_outcome_optimal)
    X1 = X_train_t[X_train_t['T'] == 1]['X1']
    X2 = X_train_t[X_train_t['T'] == 1]['X2']
    E_y_T0 = X_train_t[X_train_t['T'] == 1]['Y']
    EXPECTED_S_XGB = {percent: true_expected_outcome_S_learner_XGB}
    EXPECTED_T_XGB = {percent: true_expected_outcome_T_learner_XGB}
    EXPECTED_TRUE = {percent: expected_outcome_true_policy}
    EXPECTED_random = {percent: expected_outcome_random_policy}
    EXPECTED_optimal = {percent: true_expected_outcome_optimal}
    EXPECTED_zero = {percent: expected_outcome_zero_policy}
    
    rmse_S = {percent: rmse_tau_1_0_S_learner_XGB}
    rmse_T = {percent: rmse_tau_1_0_T_learner_XGB}
    
    shared_state_actor.update.remote(percent, EXPECTED_S_XGB[percent], EXPECTED_T_XGB[percent],EXPECTED_TRUE[percent],EXPECTED_random[percent],EXPECTED_optimal[percent],
                                     EXPECTED_zero[percent],rmse_S[percent],rmse_T[percent])
    ray.get(result_actor.update.remote(percent, expected_outcome_S_learner_XGB))
    return expected_outcome_S_learner_XGB, expected_outcome_T_learner_XGB


dataset_directory = A DEFINIR

for filename in tqdm(os.listdir(dataset_directory), desc="Traitement des fichiers CSV", unit="fichier"):
    if filename.endswith(".csv") and filename not in exclusions:
    #if filename.endswith(".csv"):  # Vérifie si le fichier est un fichier CSV
        print(filename)
        result_actor = ResultActor.remote()
        shared_state = SharedState.remote()
        file_path = os.path.join(dataset_directory, filename)
        data = pd.read_csv(file_path)

        percentages = list(range(0, 101, 5))
        futures = [iteration.remote(data,i,shared_state,nb_per_t=5000) for i in percentages]
        results = ray.get(futures)
        DICT8_EXPECTED = ray.get(result_actor.get_data.remote())
        EXPECTED_S_XGB_total = ray.get(shared_state.get_expected_s_xgb.remote())
        EXPECTED_T_XGB_total = ray.get(shared_state.get_expected_t_xgb.remote())
        EXPECTED_TRUE = ray.get(shared_state.get_expected_true.remote())
        EXPECTED_random = ray.get(shared_state.get_expected_random.remote())
        EXPECTED_optimal = ray.get(shared_state.get_expected_opti.remote())
        EXPECTED_zero = ray.get(shared_state.get_expected_zero.remote())
        
        
        rmse_S = ray.get(shared_state.get_rmse_S.remote())
        rmse_T = ray.get(shared_state.get_rmse_T.remote())
        
        sorted_EO_S_XGB = sorted(EXPECTED_S_XGB_total.keys())
        sorted_EO_S_XGB = [EXPECTED_S_XGB_total[key] for key in sorted_EO_S_XGB]
        sorted_EO_T_XGB = sorted(EXPECTED_T_XGB_total.keys())
        sorted_EO_T_XGB = [EXPECTED_T_XGB_total[key] for key in sorted_EO_T_XGB]
        
        sorted_keys_TRUE = sorted(EXPECTED_TRUE.keys())
        sorted_keys_TRUE = [EXPECTED_TRUE[key] for key in sorted_keys_TRUE]
        
        sorted_keys_random = sorted(EXPECTED_random.keys())
        sorted_keys_random = [EXPECTED_random[key] for key in sorted_keys_random]
        
        sorted_keys_optimal = sorted(EXPECTED_optimal.keys())
        sorted_keys_optimal = [EXPECTED_optimal[key] for key in sorted_keys_optimal]
        
        sorted_keys_zero = sorted(EXPECTED_zero.keys())
        sorted_keys_zero = [EXPECTED_zero[key] for key in sorted_keys_zero]
        
        sorted_keys_average = sorted(EXPECTED_average.keys())
        sorted_keys_average = [EXPECTED_average[key] for key in sorted_keys_average]
        
        
        pourcentages = list(range(0, 101, 5))
               
        sorted_keys_S_XGB = sorted(rmse_S.keys())
        sorted_keys_S_XGB = [rmse_S[key] for key in sorted_keys_S_XGB]
        
        sorted_keys_T_XGB = sorted(rmse_T.keys())
        sorted_keys_T_XGB = [rmse_T[key] for key in sorted_keys_T_XGB]
        
        pourcentages = list(range(0, 101, 5))
               
        results_folder = 'Results_XL_XGB_RF'  # ou 'chemin/vers/le/dossier/Results'
        
        if not os.path.exists(results_folder):
            os.makedirs(results_folder)
        
        parts = filename.split('_')
        nb_t = parts[0]  # Le nombre avant le tiret
        suffix = parts[1] if len(parts) > 1 else ""  # Le suffixe après le tiret
        
        if suffix.startswith('01'):
            condition = 'commence par 01'
            data_uplift='01'
            fold = filename.split("01", 1)[1]
            fold = fold.rstrip('.csv')
        elif suffix.startswith('0406'):
            condition = 'commence par 0406'
            data_uplift='0406'
            fold = filename.split("0406", 1)[1]
            fold = fold.rstrip('.csv')
        elif suffix.startswith('0208'):
            condition = 'commence par 0208'
            data_uplift='0208'
            fold = filename.split("0208", 1)[1]
            fold = fold.rstrip('.csv')
        else:
            condition = 'ne correspond à aucune condition'
        
        df = pd.DataFrame(columns=["Methode", "Data uplift", "Nb ind par t","Nb de t","Nb case uplift","Fold","EO test","EO test list",
                                   "RMSE test","RMSE test list"])
        
        new_row = {
            "Methode": "X Learner XGB",
            "Data uplift": data_uplift,  
            "Nb ind par t": 5000,  
            "Nb de t": nb_t,  
            "Nb case uplift": 20,  
            "Fold" : fold,
            "EO test":sum(sorted_EO_S_XGB) / len(sorted_EO_S_XGB),
            "EO test list":sorted_EO_S_XGB,
            "RMSE test":sum(sorted_keys_S_XGB) / len(sorted_keys_S_XGB),
            "RMSE test list": sorted_keys_S_XGB
        
        }
        
        new_row = pd.DataFrame([new_row]) 
        df = pd.concat([df, new_row], ignore_index=True)
        
        new_row = {
            "Methode": "X Learner LR",
            "Data uplift": data_uplift,  
            "Nb ind par t": 5000,  
            "Nb de t": nb_t,  
            "Nb case uplift": 20,  
            "Fold" : fold,
            "EO test":sum(sorted_EO_T_XGB) / len(sorted_EO_T_XGB),
            "EO test list":sorted_EO_T_XGB,
            "RMSE test":sum(sorted_keys_T_XGB) / len(sorted_keys_T_XGB),
            "RMSE test list": sorted_keys_T_XGB
        
        }
        
        new_row = pd.DataFrame([new_row]) 
        df = pd.concat([df, new_row], ignore_index=True)
        titre=nb_t+'_' + data_uplift+fold+'.csv'
        csv_path = os.path.join(results_folder, titre)
        df.to_csv(csv_path, sep='\t', index=False)


Traitement des fichiers CSV:   0%|          | 0/150 [00:00<?, ?fichier/s]

3_04065.csv


2024-09-30 21:20:45,155	INFO worker.py:1770 -- Started a local Ray instance.
Traitement des fichiers CSV:   5%|▍         | 7/150 [00:34<11:49,  4.96s/fichier]

17_02086.csv


Traitement des fichiers CSV:   5%|▌         | 8/150 [05:09<2:00:08, 50.76s/fichier]

...

3_017.csv


Traitement des fichiers CSV:  27%|██▋       | 41/150 [3:03:46<3:31:07, 116.22s/fichier]

9_017.csv


Traitement des fichiers CSV:  28%|██▊       | 42/150 [3:05:24<3:19:05, 110.60s/fichier]

9_02082.csv
